## Importing the libraries

In [439]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## Importing the dataset

In [440]:
dataset = pd.read_excel("dataset.xlsx")
dataset.head()

,KOD,BARKOD,MAMUL,K.İÇİ,GR,LİSTE,FATURA ALTI %,DÖNEM SONU %,KDV,NET MALİYET,Ö.RAF,KATEGORİ,AY
0,67293883,8690637840746,1,8,400.0,21.85,12.0,5,0.01,18.449266,23.061583,FOOD,MART
1,67293891,8690637840821,1,8,400.0,21.85,12.0,5,0.01,18.449266,23.061583,FOOD,MART
2,67767533,8690637905896,1,12,600.0,27.00,13.0,5,0.01,22.538655,28.173319,FOOD,MART
3,67754288,8690637905193,1,12,600.0,27.00,13.0,5,0.01,22.538655,28.173319,FOOD,MART
4,68612788,8690637999277,1,8,335.0,39.75,10.0,5,0.01,34.326113,42.907641,FOOD,MART


In [441]:
selected_columns = ['KATEGORİ', 'AY', 'MAMUL', 'K.İÇİ', 'GR', 'LİSTE', 'NET MALİYET']
data_selected = dataset[selected_columns]
data_selected.head()

,KATEGORİ,AY,MAMUL,K.İÇİ,GR,LİSTE,NET MALİYET
0,FOOD,MART,1,8,400.0,21.85,18.449266
1,FOOD,MART,1,8,400.0,21.85,18.449266
2,FOOD,MART,1,12,600.0,27.00,22.538655
3,FOOD,MART,1,12,600.0,27.00,22.538655
4,FOOD,MART,1,8,335.0,39.75,34.326113


In [442]:
X = data_selected.drop(columns=["KATEGORİ"])  # Giriş verileri
y = data_selected["KATEGORİ"]  # Çıkış (etiket) verileri

In [443]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3820 entries, 0 to 3819
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AY           3820 non-null   object 
 1   MAMUL        3820 non-null   int64  
 2   K.İÇİ        3820 non-null   int64  
 3   GR           3820 non-null   float64
 4   LİSTE        3820 non-null   float64
 5   NET MALİYET  3820 non-null   float64
dtypes: float64(3), int64(2), object(1)
memory usage: 179.2+ KB


In [444]:
print(X.head())
print(y.head())

     AY  MAMUL  K.İÇİ     GR  LİSTE  NET MALİYET
0  MART      1      8  400.0  21.85    18.449266
1  MART      1      8  400.0  21.85    18.449266
2  MART      1     12  600.0  27.00    22.538655
3  MART      1     12  600.0  27.00    22.538655
4  MART      1      8  335.0  39.75    34.326113
0    FOOD
1    FOOD
2    FOOD
3    FOOD
4    FOOD
Name: KATEGORİ, dtype: object


In [445]:
categorical_columns = ['AY']
numeric_columns = ['K.İÇİ', 'GR', 'LİSTE']

In [446]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

# Sayısal ve kategorik sütunları işlemek için bir sütun dönüştürücü oluşturun
preprocessor = ColumnTransformer(
    transformers=[
        ("sayısal", StandardScaler(), numeric_columns),
        # ("kategori", OneHotEncoder(), categorical_columns),
    ],
    remainder="passthrough",
)

# Verileri dönüştürün
X.iloc[:, 1:] = preprocessor.fit_transform(X.iloc[:, 1:])
le = LabelEncoder()
X.iloc[:, 0] = le.fit_transform(X.iloc[:, 0])
y = le.fit_transform(y)


C:\Users\PC\AppData\Local\Temp\ipykernel_31212\2324700065.py:16: FutureWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`



In [447]:
x_dataset = pd.DataFrame(X)
x_dataset.head()

,AY,MAMUL,K.İÇİ,GR,LİSTE,NET MALİYET
0,4,-0.565928,-0.110449,-0.986225,1.0,18.449266
1,4,-0.565928,-0.110449,-0.986225,1.0,18.449266
2,4,-0.470838,0.121934,-0.829472,1.0,22.538655
3,4,-0.470838,0.121934,-0.829472,1.0,22.538655
4,4,-0.565928,-0.185974,-0.441394,1.0,34.326113


In [448]:
from sklearn.model_selection import train_test_split

# Verileri eğitim ve test kümelerine bölün
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Modeli oluşturun

In [449]:
from tensorflow import keras
from keras import layers

model = keras.Sequential([ # Sıralı bir model oluşturun
    layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]), # Giriş katmanı
    layers.Dense(64, activation='relu'), # Gizli katman
    layers.Dense(3, activation='softmax') # Çıkış katmanı
])


#### Modeli Derle

In [450]:
model.compile(
    optimizer='adam', # Optimizasyon fonksiyonu
    loss='sparse_categorical_crossentropy', # Kayıp fonksiyonu
    metrics=['accuracy'] # Başarı metriği
)

In [451]:
y_train = y_train.reshape(-1, 1)

#### Eğitim işlemini başlatın

In [452]:
history = model.fit(
    X_train, y_train, # Eğitim verileri
    validation_split=0.2, # Eğitim verilerinin %20'sini doğrulama verileri olarak kullanın
    batch_size=32, # Her iterasyonda 32 örnek kullanın
    epochs=100, # 100 iterasyon
    verbose=1 # Her iterasyonda ekrana çıktı ver
)

Epoch 1/100
77/77 [==============================] - 1s 3ms/step - loss: 0.9635 - accuracy: 0.7034 - val_loss: 0.4539 - val_accuracy: 0.8039
Epoch 2/100
77/77 [==============================] - 0s 1ms/step - loss: 0.3114 - accuracy: 0.8953 - val_loss: 0.2959 - val_accuracy: 0.8807
Epoch 3/100
77/77 [==============================] - 0s 1ms/step - loss: 0.1947 - accuracy: 0.9452 - val_loss: 0.1886 - val_accuracy: 0.9395
Epoch 4/100
77/77 [==============================] - 0s 1ms/step - loss: 0.1377 - accuracy: 0.9660 - val_loss: 0.1600 - val_accuracy: 0.9428
Epoch 5/100
77/77 [==============================] - 0s 1ms/step - loss: 0.1301 - accuracy: 0.9624 - val_loss: 0.1619 - val_accuracy: 0.9412
Epoch 6/100
77/77 [==============================] - 0s 1ms/step - loss: 0.1076 - accuracy: 0.9689 - val_loss: 0.1246 - val_accuracy: 0.9542
Epoch 7/100
77/77 [==============================] - 0s 1ms/step - loss: 0.0852 - accuracy: 0.9738 - val_loss: 0.1051 - val_accuracy: 0.9592
Epoch 8/100
7

#### Eğitim ve doğrulama kayıp grafiği

In [453]:
# Eğitim ve doğrulama kayıp grafiği ve Eğitim ve doğrulama başarı grafiği

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(y=history.history['loss'], name="Eğitim Kaybı"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=history.history['val_loss'], name="Doğrulama Kaybı"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=history.history['accuracy'], name="Eğitim Başarısı"),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(y=history.history['val_accuracy'], name="Doğrulama Başarısı"),
    row=1, col=2
)

fig.update_layout(height=500, width=800, title_text="Eğitim ve Doğrulama Metrikleri")
fig.show()



In [454]:
# Test verileri üzerinde modeli değerlendirin
y_test = y_test.reshape(-1, 1)
model.evaluate(X_test, y_test, verbose=2)

# Test verileri üzerinde tahmin yapın
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred = le.inverse_transform(y_pred)

y_test = le.inverse_transform(y_test)

from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

cm = confusion_matrix(y_test, y_pred, labels=["Food", "HC", "PC"])

fig = ff.create_annotated_heatmap(
    z=cm,
    x=["Food", "HC", "PC"],
    y=["Food", "HC", "PC"],
    colorscale='Viridis'
)

fig.update_layout(title_text='Karmaşıklık Matrisi', xaxis = dict(title='Tahmin'), yaxis = dict(title='Gerçek'))

fig.show()


24/24 - 0s - loss: 0.0244 - accuracy: 0.9935 - 35ms/epoch - 1ms/step
24/24 [==============================] - 0s 826us/step


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:155: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

